In [2]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [6]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [7]:
start_year = 1981
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)
                        

2024-10-06 17:08:31
2024-10-06 17:18:16
2024-10-06 17:30:58
2024-10-06 17:44:19
2024-10-06 17:57:52
2024-10-06 18:10:58
2024-10-06 18:24:53
2024-10-06 18:37:46
2024-10-06 18:52:09
2024-10-06 19:08:14
2024-10-06 19:22:12
2024-10-06 19:36:45
2024-10-06 19:52:25
2024-10-06 20:08:30
2024-10-06 20:23:21
2024-10-06 20:36:43
2024-10-06 20:50:05
2024-10-06 21:03:15
2024-10-06 21:18:29
2024-10-06 21:32:42
2024-10-06 21:46:10
2024-10-06 21:59:05
2024-10-06 22:13:50
2024-10-06 22:27:13


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1981
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:38:22
2024-11-02 20:48:22
2024-11-02 21:02:10
2024-11-02 21:16:17
2024-11-02 21:30:48
2024-11-02 21:43:49
2024-11-02 21:57:15
2024-11-02 22:10:06
2024-11-02 22:24:14
2024-11-02 22:37:38
2024-11-02 22:51:49
2024-11-02 23:06:09
2024-11-02 23:19:56
2024-11-02 23:34:11
2024-11-02 23:47:17
2024-11-03 00:00:22
2024-11-03 00:14:28
2024-11-03 00:26:05
2024-11-03 00:39:31
2024-11-03 00:53:28
2024-11-03 01:06:45
2024-11-03 01:20:25
2024-11-03 01:34:41
2024-11-03 01:48:57


In [4]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1981
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:03:15
2024-11-03 01:16:49
2024-11-03 01:30:25
2024-11-03 01:45:36
2024-11-03 02:00:33
2024-11-03 02:12:59
2024-11-03 02:27:02
2024-11-03 02:40:05
2024-11-03 02:53:11
2024-11-03 03:06:02
2024-11-03 03:19:38
2024-11-03 03:32:53
2024-11-03 03:46:24
2024-11-03 04:00:04
2024-11-03 04:14:06
2024-11-03 04:27:50
2024-11-03 04:42:05
2024-11-03 04:54:53
2024-11-03 05:08:18
2024-11-03 05:21:03
2024-11-03 05:33:33
2024-11-03 05:45:51
2024-11-03 05:58:35
2024-11-03 06:10:45


In [4]:
import glob
def create_temp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_tmin = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_tmean = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_tmin,ds_tmean



def create_dtemp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_dtmin = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_dtmean = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_dtmin,ds_dtmean

In [5]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern = 'T2.wrf2d_d01_????-??-??.nc'

folder_names = [

    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',

]


for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path = os.path.join(base_path, folder, file_pattern)
    all_files = glob.glob(full_path)
    #####
    summer_files = [f for f in all_files if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_summer = xr.open_mfdataset(summer_files)
    ds_jja = ds_summer.sel(time=ds_summer['time'].dt.month.isin([6, 7, 8]))
    lonn = ds_jja.lon.values
    latt = ds_jja.lat.values
    arr_t = ds_jja.t2.values

    ds_tmin,ds_tmean = create_temp_dataset(arr_t, latt, lonn)

    output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
    ds_tmin.to_netcdf(output_folder+'temp_min_'+folder+'.nc')
    ds_tmean.to_netcdf(output_folder+'temp_mean_'+folder+'.nc')

2024-11-03 21:11:46
2024-11-03 21:16:30
2024-11-03 21:21:42
2024-11-03 21:26:55
2024-11-03 21:32:13
2024-11-03 21:37:31
2024-11-03 21:42:37
2024-11-03 21:47:57
2024-11-03 21:53:31


In [3]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 1981
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:22:31
2025-02-17 01:22:53
2025-02-17 01:23:17
2025-02-17 01:23:38
2025-02-17 01:23:59
2025-02-17 01:24:18
2025-02-17 01:24:38
2025-02-17 01:24:57
2025-02-17 01:25:17
2025-02-17 01:25:39
2025-02-17 01:25:58
2025-02-17 01:26:19
2025-02-17 01:26:39
2025-02-17 01:27:01
2025-02-17 01:27:28
2025-02-17 01:27:46
2025-02-17 01:28:05
2025-02-17 01:28:27
2025-02-17 01:28:52
2025-02-17 01:29:13
2025-02-17 01:29:35
2025-02-17 01:29:53
2025-02-17 01:30:14
2025-02-17 01:30:34
2025-02-17 01:30:58
2025-02-17 01:31:28
2025-02-17 01:31:48
2025-02-17 01:32:13
2025-02-17 01:32:35
2025-02-17 01:33:02
2025-02-17 01:33:34
2025-02-17 01:34:00
2025-02-17 01:34:03
2025-02-17 01:34:29
2025-02-17 01:34:53
2025-02-17 01:35:09
2025-02-17 01:35:38
2025-02-17 01:36:02
2025-02-17 01:36:25
2025-02-17 01:36:50
2025-02-17 01:37:12
2025-02-17 01:37:47
2025-02-17 01:38:10
2025-02-17 01:38:30
2025-02-17 01:38:50
2025-02-17 01:39:12
2025-02-17 01:39:37
2025-02-17 01:40:02
2025-02-17 01:40:24
2025-02-17 01:40:55
